<a href="https://colab.research.google.com/github/aronyo24/Research-paper/blob/main/Digitalization_of_Land_Record_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf



In [ ]:
df=pd.read_csv("/content/drive/MyDrive/Research Paper/test_digi_manual_land_re.csv")
df

In [ ]:
df.columns.tolist()

In [ ]:
#df=df.drop(columns=['Unnamed: 22', 'Unnamed: 23','Unnamed: 24'])
#df

In [ ]:
df.rename(columns={
    "Service Type ":"service_type",
    'Location':'location',
    "Education": "education",
    "Phone User": "phone_type",
    "Laptop Access": "laptop_access",
    "Internet Access": "internet_access",
    "Who Submitted Application" :"who_submitted_application",
    "Device Used for submitting application": "device_used",
    "Application Submission times": "app_submission_times",
    "Cost of Submitting Application ": "app_cost",
    "Cost for e-mutation": "e_mutation_cost",
    "To whom, pay": "payment_receiver",
    "Days to get the service": "service_days",
    "Citizen Awareness of Circulars": "awareness_circulars",
    "Citizen's Awareness for Complaints": "awareness_complaints",
    "Visits to government offices": "gov_office_visits",
    "Cash transaction to govt Offices": "cash_transaction",
    "Citizen's Satistaction": "citizen_satisfaction",
    "Recommendation from elites":"recommendation_f_elites",
    "Previous experience of mutation":"Previous_experience_mut"

}, inplace=True)

df


In [ ]:
##  Education: 1 = Educated, 0 = Uneducated
df['education'] = df['education'].apply(lambda x: 0 if x == "ill" else 1)

##  Phone Type: 1 = Android, 0 = Button Phone
df['phone_type'] = df['phone_type'].apply(lambda x: 1 if x == 1 else 0)

##  Laptop Access: 1 = Yes, 0 = No
df['laptop_access'] = df['laptop_access'].apply(lambda x: 1 if x == 1 else 0)

## Internet Access: 1 = Yes, 0 = No
df['internet_access'] = df['internet_access'].apply(lambda x: 1 if x == 1 else 0)

## Citizen Awareness of Circulars: 1 = Yes, 0 = No
df['awareness_circulars'] = df['awareness_circulars'].apply(lambda x: 1 if x == 1 else 0)

## Citizen's Awareness for Complaints: 1 = Yes, 0 = No
df['awareness_complaints'] = df['awareness_complaints'].apply(lambda x: 1 if x == 1 else 0)
## Device Used for submitting application 1 = laptop ,0 = phone
df['device_used'] = df['device_used'].apply(lambda x: 1 if x == 1 else 0)

## Cash transaction to govt Offices 1 = yes ,0 = no
df['cash_transaction'] = df['cash_transaction'].apply(lambda x: 1 if x == 1 else 0)

##Citizen's Satistaction 1= yes , 0 = no
df['citizen_satisfaction'] = df['citizen_satisfaction'].apply(lambda x: 1 if x == 1 else 0)



In [ ]:
df.describe()

In [ ]:
df = df.dropna()

df


In [ ]:
df.hist(figsize=(15, 13), bins=20)
plt.show()

In [ ]:
df[['e_mutation_cost','service_days']]
# in column e mutation cost means money range 1= 1170, 2 = 1170 to 2000 , 3= 2001 to 3000 & 4= 3001 to 4000
# in column service days means m range 1= 28 ,2 = 28 to 45



# Define the mappings
e_mutation_cost_map = {
    1: 1170,
    2: 1585,
    3: 2500,
    4: 3500
}

service_days_map = {
    1: 28,
    2: 36
}

# Map values for e_mutation_cost if they are in [1, 2, 3, 4]
df['e_mutation_cost'] = df['e_mutation_cost'].apply(lambda x: e_mutation_cost_map.get(x, x))

# Map values for service_days if they are in [1, 2]
df['service_days'] = df['service_days'].apply(lambda x: service_days_map.get(x, x))

df


In [ ]:
df.hist(figsize=(15, 13), bins=20)
plt.show()

In [ ]:
# Define Treatment (Service Type: 1 = Digital, 0  Manual)
df['Treatment'] = df['service_type']

# Define Post-digitalization period (Assume service_days > 20 means post)
df['Post'] = (df['service_days'] <= 20).astype(int)

# DID Interaction Term
df['Treatment_Post'] = df['Treatment'] * df['Post']

In [ ]:
model = smf.ols("service_days ~ Treatment + Post + Treatment_Post + education + internet_access + laptop_access", data=df).fit()
model.summary()

In [ ]:
model2 = smf.ols("citizen_satisfaction ~ Treatment + Post + Treatment_Post + education + internet_access + laptop_access", data=df).fit()
model2.summary()

In [ ]:
sns.barplot(x="Treatment_Post", y="service_days", data=df, palette="coolwarm")
plt.title("Impact of Digitalization on Service Time")
plt.xlabel("Digital Service (1=Yes, 0=No)")
plt.ylabel("Days to Get the Service")
plt.show()

In [ ]:
sns.boxplot(x="Post", y="app_cost", data=df, palette="pastel")
plt.title("Impact of Digitalization on Cost of Application")
plt.xlabel("Post-Digitalization (1=Yes, 0=No)")
plt.ylabel("Application Cost (Taka)")
plt.show()


In [ ]:
sns.countplot(x="citizen_satisfaction", hue="Treatment", data=df, palette="coolwarm")
plt.title("Citizen Satisfaction: Digital vs. Manual Service")
plt.xlabel("Citizen Satisfaction (1=Yes, 0=No)")
plt.ylabel("Count")
plt.legend(["Manual", "Digital"])
plt.show()


In [ ]:
#Impact of Internet Access on Service Time
plt.figure(figsize=(8, 5))
sns.boxplot(x="internet_access", y="service_days", data=df, palette="coolwarm")
plt.title("Impact of Internet Access on Service Time", fontsize=14)
plt.xlabel("Internet Access (1=Yes, 0=No)")
plt.ylabel("Days to Get the Service")
plt.show()

In [ ]:
#effect of Internet Access on E-Mutation Cost

sns.histplot(x='internet_access',y="e_mutation_cost",data=df,palette='viridis')
plt.title("Effect of Internet Access on E-Mutation Cost",fontsize=14)
plt.xlabel("Internet Access (1=Yes, 0=No)")
plt.ylabel("E-Mutation Cost (in Taka)")
plt.show()

In [ ]:
#Citizen Satisfaction

sns.barplot(x='education',y='citizen_satisfaction',ci=None,data=df,palette='magma')
plt.title("Citizen Satisfaction Based on Education", fontsize=14)
plt.xlabel("Education Level (1=Educated, 0=Uneducated)")
plt.ylabel("Satisfaction Level (1=Yes, 0=No)")
plt.show()

In [ ]:
#Cash Transaction and Awareness of Complaints
sns.countplot(x="cash_transaction", hue="awareness_complaints", data=df, palette="Set2")
plt.title("Cash Transaction and Awareness of Complaints", fontsize=14)
plt.xlabel("Cash Transaction (1=Yes, 0=No)")
plt.ylabel("Count")
plt.legend(["Not Aware", "Aware"])
plt.show()

In [ ]:
# Define Treatment (Service Type: 1 = Digital, 0  Manual)
df['Treatment'] = df['service_type']

# Define Post-digitalization period (Assume service_days > 20 means post)
df['Post'] = (df['service_days'] <= 20).astype(int)

# DID Interaction Term
df['Treatment_Post'] = df['Treatment'] * df['Post']


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Data for visualization
systems = ['Manual (Before)', 'Manual (After)', 'Digital (Before)', 'Digital (After)']
times = [90, 60, 45, 15]  # Hypothetical averages

# Plot the bar chart
plt.figure(figsize=(7,5))
sns.barplot(x=systems, y=times, palette=['red', 'orange', 'blue', 'green'])

# Labels
plt.ylabel("Service Time (Days)")
plt.xlabel("System Type")
plt.title("Impact of Digitalization on Service Time")

plt.show()


In [ ]:
import pandas as pd

# Sample Data
data = {
    "System": ["Manual"] * 40 + ["Digital"] * 40,
    "Service_Time": np.random.normal(80, 10, 40).tolist() + np.random.normal(20, 5, 40).tolist()  # Random Data
}

df = pd.DataFrame(data)

# Box Plot
plt.figure(figsize=(6,5))
sns.boxplot(x="System", y="Service_Time", data=df, palette=['red', 'blue'])
plt.ylabel("Days to Get Service")
plt.title("Service Time Distribution: Manual vs. Digital")
plt.show()
